## Imports

In [24]:
import os
import json
import ipywidgets as widgets
from ipywidgets import Text, HTML, HBox, Layout, VBox
from ipyleaflet import (
    Map,
    Marker,
    TileLayer, ImageOverlay,
    Polyline, Polygon, Rectangle, Circle, CircleMarker,
    GeoJSON,
    DrawControl,
    WidgetControl,
    basemaps,
    Popup,
    Icon,
    MarkerCluster
)

from math import sqrt

## Slice Manager

In [25]:
from fabrictestbed.slice_manager import SliceManager, Status

credmgr_host = os.environ['FABRIC_CREDMGR_HOST']
orchestrator_host = os.environ['FABRIC_ORCHESTRATOR_HOST']
print(f"CM Host: {credmgr_host} Orchestrator Host: {orchestrator_host}")

slice_manager = SliceManager(oc_host = orchestrator_host, cm_host = credmgr_host, project_name='all', scope='all')
slice_manager.initialize()

print("Token location: {}".format(slice_manager.token_location))
print()

#Cofigure SSH Key
ssh_key = None
with open ("/home/fabric/.ssh/id_rsa.pub", "r") as myfile:
    ssh_key=myfile.read().strip()

    
try:
    id_token, refresh_token = slice_manager.refresh_tokens()
except Exception as e:
    print("Exception occurred while getting tokens:{}".format(e))

fabric_refresh_token=slice_manager.get_refresh_token()
print()
print("New Refresh Token: {}".format(fabric_refresh_token))
print()
print("Stored new Refresh Token")
%store fabric_refresh_token
print()

CM Host: beta-2.fabric-testbed.net Orchestrator Host: beta-7.fabric-testbed.net
Token location: /home/fabric/.tokens.json

Exception occurred while getting tokens:"(invalid_grant) invalid refresh token"


New Refresh Token: NB2HI4DTHIXS6Y3JNRXWO33OFZXXEZZPN5QXK5DIGIXTEODGG5QTOOBYMUZDEYZUG4YDOY3CGNQTGNRXGM4GIMRXMU2TQMR7OR4XAZJ5OJSWM4TFONUFI33LMVXCM5DTHUYTMMRZGM4DMNJUG42TKMBGOZSXE43JN5XD25RSFYYCM3DJMZSXI2LNMU6TQNRUGAYDAMBQ

Stored new Refresh Token
Stored 'fabric_refresh_token' (str)



## Get advertised topology and sites

In [26]:
from fabrictestbed.slice_editor import Capacities

status, advertised_topology = slice_manager.resources()

site_dict = {}

#query the resource topology and store longitude/latitude of the objects in dictionary
for t in advertised_topology.sites.values():
    name = t.name 
    loc_info = t.get_property("location")
    if loc_info is not None:
        loc = loc_info.to_latlon()
        site_dict[name] = loc

SliceManagerException: "(invalid_grant) invalid refresh token"


## Create experiment with map

In [19]:
from fabrictestbed.slice_editor import ExperimentTopology, Capacities, ComponentType, LinkType, Layer

# Create Experiment Topology
experiment = ExperimentTopology()

slice_name = "MapDemoSlice"

## Functions

In [20]:
node_name = widgets.Text(
    value = '',
    placeholder='enter name',
    description='Node Name:',
    disabled=False
)

site_option = widgets.Text(
    value='UKY',
    description='Site:',
    disabled=False
)

core_slider = widgets.IntSlider(min=2, max=32, step=2, value=4, description='Cores')
ram_slider = widgets.IntSlider(min=2, max=64, step=2, value=16, description='RAM')
disk_slider = widgets.FloatLogSlider(base=2, min=5, max=17, step=1, value=128,description='Disk')

img_ref_select = widgets.Select(
    options=['qcow2', 'qchicken2', 'qpig2'],
    value='qcow2',
    description='Image Type:',
    disabled=False
)

img_type_select = widgets.Select(
    options=['default_ubuntu_20', 'default_arch_20', 'default_kali_21'],
    value='default_ubuntu_20',
    description='Image Reference:',
    disabled=False
)


def id_node_components(self, component_list, type_list, node):
    for t in advertised_topology.sites.values():
         if t.name == node:
            for c in t.components.values():
                component_list.append(c.name)
                type_list.append(c.get_property('type'))

## Node Creation Map
Map where you click on the markers to make nodes, add components and links.

In [21]:
%%html 
<style> 
.leaflet-div-icon { background-color: transparent; border-color: transparent; font-size: medium;}
.jupyter-widgets.jp-OutputArea-output {overflow: auto}
</style>

In [23]:
from ipyleaflet import DrawControl, CircleMarker, AwesomeIcon, DivIcon
import time
import functools


#initialize map 
map_center = (38.12480976137421, -95.7129)
canvas = Map(basemap = basemaps.Esri.WorldStreetMap, center = map_center, zoom = 3.8)

#helper container/vars
markers = {}
nodes = {}
clusters = {}
for name in site_dict.keys():
    nodes[name] = []

circles = {}
add_btns = {}
layers = []


#NODE COMPONENT TYPES LIST
node_component_types = []
        
for t in ComponentType:
    node_component_types.append(t)

#FUNCTION TO FIND CLOSEST SITE GIVEN COORDINATES
def find_closest(coordinates, sitedict):
    closest_site = 'UKY'
    closest_value = 500000
    for site,coors in sitedict.items():
        distance = sqrt(pow((coordinates[1] - coors[0]), 2) + pow((coordinates[0] - coors[1]), 2))
        if distance < closest_value:
            closest_site = site
            closest_value = distance
    return closest_site


#DRAW CONTROL - TO DRAW LINKS?
dc = DrawControl(circlemarker = {}, polygon= {})

feature_collection = {
            'type': 'FeatureCollection',
            'features': []} 

def handle_draw(self, action, geo_json):
    if action in ['created', 'edited']:
        feature_collection['features'].append(geo_json)
        if feature_collection['features'][-1]['geometry']['type'] == 'LineString':
            coor1, coor2 = feature_collection['features'][-1]['geometry']['coordinates'][0]
            coor3, coor4 = feature_collection['features'][-1]['geometry']['coordinates'][1]
            
            start = find_closest( (coor1, coor2), site_dict)
            finish = find_closest((coor3, coor4), site_dict)
            header_text = '<b>New Link</b><br>' 
            links_text = 'Between ' + start + ' and ' + finish + '<br>'
            links_text = links_text + '- dashboard to create new link -'
            
            def cancel_link(self):
                canvas.remove_control(linkfo)
               #TODO: REMOVE THE DRAWN LINK
                
            exit = widgets.Button(
                    description='Cancel',
                    disabled=False,
                    tooltip='click to cancel',)

            exit.on_click(cancel_link)
            
            
            b = VBox([HTML(header_text),HTML(links_text), exit])
            b.layout = Layout(overflow_y= 'auto', padding = "4px")
         
        
            linkfo = WidgetControl(widget = b, position='bottomright')
            canvas.add_control(linkfo)

            
    elif action == 'deleted':
        feature_collection['features'].remove(geo_json)
        
    return

dc.on_draw(handle_draw)
canvas.add_control(dc)

#HEADER
w, h = 150,10
html = '<b><u>Visual Slice Editor</u></b>'
transp_icon = DivIcon(html=html, bg_pos=(0, 0), icon_size=[w, h])
header_loc = (49.5, -98.181852)
header_mark = Marker(location=header_loc, icon=transp_icon, draggable=True)
canvas.add_layer(header_mark)

#USER CHOOSES SLICE NAME
slice_name_box = widgets.Text(
    value = '',
    placeholder='Enter slice name',
    disabled=False
)

slice_name_control = WidgetControl(widget = slice_name_box, position = 'topright')
canvas.add_control(slice_name_control)


#INIALIZE NODE CREATION DASHBOARD
init = [HTML('Click on a site to create a new node!')]
my_layout = Layout(height='50%', overflow_y= 'auto', padding = "4px" )
html_box = VBox(init , layout = my_layout )
control = WidgetControl(widget = html_box, position='topright')
canvas.add_control(control)



#NOTE: Change path to local
#icon = Icon(icon_url='https://jupyter-beta.fabric-testbed.net/user/nannkat@bu.edu/files/jupyter-examples/Nanna/Maps/circle.png?_xsrf=2%7C416d757a%7Cab507567f51333d5f04bb4523d3fa83e%7C1628182956', icon_size=[15,15])
icon = Icon(icon_url='https://jupyter-beta.fabric-testbed.net/user/lockej@bu.edu/files/jupyter-examples/jack_test_folder/demo/marker.png?_xsrf=2%7Cae07b368%7C087b3192c3b141ab371bfb4642036d8e%7C1627646378', icon_size=[15,15])

#DELETE ONE NODE
def delete_node(self, nodename = ''):
    
    sitename = experiment.nodes[nodename].get_property('site')
    curr_node = experiment.nodes[nodename]
    
    for s, n_list in nodes.items():
        if curr_node in n_list:
            n_list.remove(curr_node)

    experiment.remove_node(nodename)
    
    m =  markers[nodename]
    
    if sitename in clusters.keys():
        cluss = clusters[sitename]
        markers_all = list(cluss.markers)
        
        for mark in markers_all:
            if mark is  m:
                markers_all.remove(mark)
                
        canvas.remove_layer(cluss)
        layers.remove(cluss)
        
        if len(markers_all) == 1:
            last = markers_all[0]
            clusters.pop(sitename)
            plus = add_btns.pop(sitename)
            canvas.remove_layer(plus)
            canvas.add_layer(last)
        else:
            newclus = MarkerCluster(markers = tuple(markers_all))
            newclus.on_click(new_node)
            layers.append(newclus)
            clusters[sitename] = newclus
            canvas.add_layer(clusters[sitename])
    else:
        canvas.remove_layer(m)
                
    markers.pop(nodename)  

#CLEAR (DELETE) ALL NODES
def delete_all(self):
    
    for node in experiment.nodes.values():
        experiment.remove_node(node.name)
    
    for l in layers:
        canvas.remove_layer(l)
        
    layers.clear()
        
    clusters.clear()
    markers.clear()
    nodes.clear()
    
    for name in site_dict.keys():
        nodes[name] = []
    

#delete all button and widget control
delete_all_nodes = widgets.Button(
    description = 'Clear All',
    disabled = False,
    tooltip ='click to remove all nodes', layout = Layout(border = 'solid 2px yellow') )

delete_all_nodes.style.button_color = 'white'

delete_all_nodes.on_click(delete_all)

delete_all_btn = WidgetControl(widget = delete_all_nodes, position='bottomright')



#SUBMIT/PREVIEW SLICE
#preview box
make_request =  widgets.Button(
        description = 'Submit',
        disabled = False,
        tooltip ='click to finalize slice request')

cancel_btn = widgets.Button(
        description = 'Cancel',
        disabled = False,
        tooltip ='click to cancel slice request')

def prev(self):
    
    #Create html box
    if slice_name_box.value == '':
        preview_content = '<center><b>Slice:</b> Untitled</center>'
    else:
        preview_content = '<center><b>Slice:</b> ' + slice_name_box.value + '</center>'
    
    preview_content += '<b>Nodes:</b><br>'
    
    if experiment.nodes:
        for node in experiment.nodes.values():
            preview_content += node.name +  '<br>'
            preview_content += '&emsp;Site: ' + node.get_property('site') + '<br>'
            preview_content += '&emsp;Capacities: '
            preview_content += 'core: ' + str(node.get_property('capacities').core) + ', '
            preview_content += 'ram: ' + str(node.get_property('capacities').ram) + ' G, '
            preview_content += 'disk: ' + str(node.get_property('capacities').disk) + ' G<br>'
            
            preview_content += '&emsp;Components:<br>'
            for component in node.components.values():
                preview_content += '&emsp;&emsp;' + component.name + ' (' + str(component) + ') <br>'
                
    
    #preview_content += '<br>'
    preview_content += '<b>Links:</b><br>'
    preview_content += '&emsp;<i>TBD</i>'
    preview_btns = HBox([VBox([make_request]), VBox([cancel_btn])])
    #layout = Layout(height = '400px', overflow_y = 'auto')
    preview_box = VBox([HTML(preview_content), preview_btns], layout = Layout(overflow_y = 'auto', height = '400px') )

    
    #Launch html box 
    preview = WidgetControl(widget = preview_box, position='bottomleft')
    
    def sub(self):
        slice_graph = experiment.serialize()
        
        status, reservations = slice_manager.create(slice_name = slice_name, slice_graph = slice_graph, ssh_key = ssh_key)
        
        canvas.remove_control(preview)
        
        res_progress = widgets.FloatProgress(value=0, min=0, max=10.0, description='Configuring..', bar_style='success', style={'bar_color': 'green'},
            orientation='horizontal')
        
        progr_bar = WidgetControl(widget = res_progress, position = 'bottomleft')
        
        success_notification = Popup(
            location=(38.12480976137421, -95.7129),
            child= HTML(''),
            close_button=True,
            auto_close=True,
            close_on_escape_key=False)
        
        configured = False
        
        if status == Status.OK:
            canvas.add_control(progr_bar)
            slice_id = reservations[0].slice_id
            state = None
            
            failed = False
            
            while state != 'StableOK':
                if res_progress.value == 10:
                    #notify slice failed
                    failed = True
                    break
                    
                status, slice_status = slice_manager.slice_status(slice_id=slice_id)
                state = slice_status.slice_state
                res_progress.value += 1
                time.sleep(20)
            if not failed:    
                configured = True
        else:
            success_notification.child = HTML('Request failed!')
        
        #Ideally get slice status and reservation time????
        
        canvas.remove_control(progr_bar)
        
        if configured:
            success_notification.child = HTML('Sucessfully reserved slice!')
        else:
            success_notification.child = HTML('Reservation failed!')
            
        canvas.add_layer(success_notification)
        
    make_request.on_click(sub)

    def cancel_reservation(self):
        canvas.remove_control(preview)
    
    cancel_btn.on_click(cancel_reservation)
    
    canvas.add_control(preview)
  
    
submit_request = widgets.Button(
    description = 'Submit/Preview',
    disabled = False,
    tooltip ='click to preview/submit',
    layout = Layout(border = 'solid 2px yellow'))

submit_request.style.button_color = 'white'
submit_request.on_click(prev)

submit_request_btn = WidgetControl(widget = submit_request, position='bottomright')

canvas.add_control(submit_request_btn)
canvas.add_control(delete_all_btn)


#ADD NODES

def add_node(self):
    
    node_name_taken_popup = Popup(
    location=map_center,
    child=HTML('Cannot add node! Node name already taken. Please choose another name and try again.'),
    close_button=True,
    auto_close=False,
    close_on_escape_key=False)

    #get values from widgets
    site = site_option.value
    
    #check if node name is taken
    for node_l in nodes.values():
        for n in node_l:
            if node_name.value == n.name:
                canvas.add_layer(node_name_taken_popup)
                layers.append(node_name_taken_popup)
                return
        
    n = experiment.add_node(name=node_name.value, site=site)
    cap = Capacities().set_fields(core=core_slider.value, ram=ram_slider.value, disk=disk_slider.value)
    n.set_properties(capacities=cap)
    
    #add node to node dict
    nodes[site].append(n)
    
    #delete node button
    delete_node_request = widgets.Button(
        description='Delete Node',
        disabled = False,
        tooltip='click to remove node',)

    delete_node_request.on_click(functools.partial(delete_node, nodename = node_name.value))
    
    #edit node button
    edit_node_request = widgets.Button(
        description='Edit',
        disabled = False,
        tooltip='click to edit node',)
    
    add_components_button = widgets.Button(
        description='Add Components',
        disabled=False,
        tooltip='Click to Add Components',)
    
    #add marker, popup and delete button
    marker = Marker(location = site_dict[site], draggable = False, title = node_name.value)
    html_str = '<b>' + node_name.value + '</b> at ' + site_option.value + '<br>' 
    html_str = html_str + 'type: VM<br>' 
    html_str = html_str + 'core: ' + str(core_slider.value) + ', '
    html_str = html_str + 'ram: ' + str(ram_slider.value) + ' G, ' 
    html_str = html_str + 'disk: ' + str(disk_slider.value) + ' G '
    html_str = html_str + '<br>'
    html_str = html_str + 'components:<br>'
    
    #info_layout = Layout(overflow_y= 'auto', padding = "4px")
    vertical_layout = Layout(overflow_y = 'auto', height = '500px', padding = "4px")
    
    modify_btns = HBox([VBox([edit_node_request]), VBox([delete_node_request])])
    
    html_content = HBox([VBox([HTML(html_str), add_components_button, modify_btns])], layout = vertical_layout)

    marker.popup = html_content
    markers[node_name.value] = marker
    
    node_components = []
    
    def add_components(self, info_str = ''):
        id_node_components(self, node_components, node_component_types, site_option.value)
        
        def submit_component(self, info_str = ''):
            comp_type = None

            html_box.children = init

            if component_select.value != '':
                typ = component_select.value.split('-')[0]
                mod = component_select.value.split('-')[1]
            
            for i, t in enumerate(node_component_types):
                if str(t) == typ:
                    comp_type = t

            c = n.add_component(ctype = comp_type, model = mod, name = component_name.value)
            component_marker = markers[n.name]
            canvas.remove_layer(component_marker)
            layers.remove(component_marker)
            
            
            info_str += '&emsp; <i>' + c.name + '</i> ('
            info_str += str(c) + ')<br>'
            
            html_content = HBox([VBox([HTML(info_str), add_components_button, modify_btns])], layout = vertical_layout)
            
            component_marker.popup = html_content
            canvas.add_layer(component_marker)
            layers.append(component_marker)
            

        submit_component_button = widgets.Button(
                description = 'Add Component',
                disabled = False,
                tooltip ='Click to submit component')

        submit_component_button.on_click(functools.partial(submit_component, info_str = info_str))
        
        component_name = widgets.Text(
            placeholder='enter name',
            description= ('comp.name'),
            disable = False)

        component_select = widgets.Select(
            options = node_components,
            value = node_components[0],
            description=('comp.model'),
            disabled = False)
        
        content_w_components = [VBox([HTML('Add component to <b>' + n.name + '</b>'), component_name, component_select, submit_component_button])] 
        #html_content.children = content_w_components
        html_box.children = content_w_components


    add_components_button.on_click(functools.partial(add_components, info_str = html_str))
    
    #plus button:
    add_icon = AwesomeIcon(name='plus', marker_color='green', icon_color='white',spin=False)
    coor1, coor2 = site_dict[site]
    add_more_marker = Marker(location = (coor1+0.25, coor2) , draggable = False, title = 'add more', icon = add_icon)
    add_more_marker.on_click(new_node)
    
    #Add marker or marker cluster if more than one on each site
    if len(nodes[site]) > 1:
        marks = []
        
        for i, node in enumerate(nodes[site]):
            marks.append(markers[node.name])
            
            if i == 0 and len(nodes[site]) == 2:
                canvas.remove_layer(markers[node.name])
                layers.remove(markers[node.name])
                
                #marks.append(add_more_marker)
                add_btns[site] = add_more_marker
                canvas.add_layer(add_more_marker)
                layers.append(add_more_marker)
          
        if len(nodes[site]) > 2:
            canvas.remove_layer(clusters[site])
            layers.remove(clusters[site])
        
    
        marker_clus = MarkerCluster(markers = tuple(marks))
        clusters[site] = marker_clus
        canvas.add_layer(marker_clus)
        layers.append(marker_clus)
       
        
    else:
        canvas.add_layer(marker)
        layers.append(marker)

    node_name.value = ''
    html_box.children = init
    core_slider.value = 4
    ram_slider.value = 16
    disk_slider.value = 128
    

add_node_request = widgets.Button(
    description='Add Node',
    disabled=False,
    tooltip='click to build node',
)

add_node_request.on_click(add_node)

#cancel btn 
def esc(self):
    html_box.children = init
    node_name.value = ''
    canvas.remove_layer(markers[site_option.value])
    
exit_btn = widgets.Button(
    description='Cancel',
    disabled=False,
    tooltip='click to exit',)

exit_btn.on_click(esc)

#NODE CREATION MENU
def node_dashboard(site_name):
    header = HTML('<center><b>New Node</b></center>')
    site_option.value = site_name
    
    btns = HBox([VBox([add_node_request]), VBox([exit_btn])])
    content = [header, node_name, site_option, core_slider, ram_slider, disk_slider, btns]
    html_box.children = content

    
#NODE CREATION ON CLICK   
def new_node(**kwargs):
    coors = kwargs['coordinates']
    name = find_closest((coors[1],coors[0]), site_dict)
    node_dashboard(name)
    

#POPULATE MAP
for name, coordinates in site_dict.items():
    m = Marker(location = coordinates, icon = icon, draggable = False, title = name)
    circles[name] = m
    m.on_click(new_node)
    canvas.add_layer(m)

canvas

Map(center=[38.12480976137421, -95.7129], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_…

## Query/Print currently added nodes // delete all nodes manually

In [ ]:
for node in experiment.nodes.values():
    print(node)
    for com in node.components.values():
        print(com)

In [ ]:
for node in experiment.nodes.values():
    experiment.remove_node(node.name)

In [ ]:
for layer in layers:
    print(layer)
    print()

In [ ]:
for marker in markers.values():
    print(marker)
    print()

In [ ]:
for node in nodes.values():
    for n in node:
        print(n)
        print()  

In [ ]:
node3 = experiment.add_node(name='node3', site='UKY')

node3_capacity = Capacities()
node3_capacity.set_fields(core=8, ram=64, disk=500)
node3.set_properties(capacities=node3_capacity, image_type='qcow2', image_ref='default_ubuntu_20')

node3_nvme1 = node3.add_component(ctype=ComponentType.NVME, model='P4510', name='n3_nvme1')

In [ ]:
def get_slices(states):
    status, slices_all = slice_manager.slices(state = 'All')
    if states[0] == 'All':
        return status, slices_all
    slices = list(filter(lambda x: x.slice_state in list(states), slices_all))
    return status, slices

## Make reservation - The old way

In [ ]:
#serialize into slice graph object
slice_graph = experiment.serialize()

#communicate with orchestrator for slice
status,reservations = slice_manager.create(slice_name = slice_name, slice_graph = slice_graph, ssh_key = ssh_key)

#get the slice id of the newly created slice and check status to see if it went through
#print("Request {}".format(status))
#print("Reservations: {}".format(reservations))

slice_id = reservations[0].slice_id

#get only slice from id and status
request, state = slice_manager.slice_status(slice_id = slice_id)
print("Slice status request {}".format(request))
print("Slice status {}".format(state))

In [ ]:
#status, slices = get_slices(states = ('Configuring', 'StableOK'))
status, slices = slice_manager.slices(state = 'All')

print("Response Status {}".format(status))
print("Slices {}".format(slices))

In [ ]:
import time

state = None
while state != 'StableOK':
    status, slice_status = slice_manager.slice_status(slice_id=slice_id)
    state = slice_status.slice_state
    print("Response Status {}".format(status))
    print("Slice State: {}".format(slices[-1].slice_state))
    time.sleep(20)

#final status
status, slice_status = slice_manager.slice_status(slice_id=slice_id)
print("Response Status {}".format(status))
print("Slice Status {}".format(slice_status)) 